In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

 # Students Performance in Exams
 ## Data Impact Assessment - Tecnologie Digitali e Societa'
 ---
 ### Collaboratori (in ordine alfabetico):
 ##### Margaret Casaletto
 ##### Ambra Destino
 ##### Angelo Laudani
 ##### Flavio Emanuele Cannavo'
 ##### Giuseppe Bartolomeo

 # About the Dataset
 Il dataset contiene 1000 osservazioni con le seguenti colonne:
 * **gender**: male/female
 * **race/ethnicity**: cinque differenti gruppi (fittizi)
 * **parental level of education**:
     (in decreasing order) masters. bachelors, associate, some college, high school, some high school.
     The "some" means they didn't finished.
 * **lunch**: standard/free or reduced
 * **test preparation course**: none/completed
 * **math score**
 * **reading score**
 * **writing score**

 # Data Exploration
 In questa tesina verranno analizzati i seguenti aspetti:
 * **Ripartizione demografica**

     Proveremo a verificare se i gruppi rappresentati nel dataset sono distribuiti equamente per etnia e genere.
     
     Gli studenti possono usufruire di pasti gratis o ridotti attraverso la partecipazione a cosiddetti *Federal Assistance Programs*  o in base alla loro condizione di senza tetto, immigrati, rifugiati, o in affidamento. A questi casi particolari si aggiungono anche il reddito familiare a la grandezza della famiglia (https://www.fns.usda.gov/nslp/nslp-fact-sheet). Sarà dunque importante capire come il tipo di pasto influisce sui risultati ottenuti.
     
     Proveremo a capire come impatta sui risultati finali il genere, l'etnia, l'educazione dei genitori, e il completamento della preparazione al test. Inoltre vedremo come la preparazione al test venga completata oppure no al variare dei gruppi, dei generi, e dell'educazione dei genitori.


 * **Criterio di sufficienza**

     Proveremo a verificare se la ripartizione corrente rispetta il criterio di sufficienza


 * **Criterio di separazione**

     Proveremo a verificare se la ripartizione corrente rispetta il criterio di separazione



 # Fattori di rischio
 Proveremo a identificare i fattori di rischio intrinsechi e a proporre dei possibili interventi


# Data Classification

Proveremo a costruire un modello statistico (classificatore) che ci aiuti a predire se uno studente avrà una valutazione sufficiente o meno

---



# Data Exploration

In [ ]:
# import and loading
df = pd.read_csv("../input/StudentsPerformance.csv")

### Diamo uno sguardo al dataset dopo qualche piccola modifica:

In [ ]:
# data cleaning
df.rename(columns={"race/ethnicity": "ethnicity", 
"parental level of education":"parent_education",
"test preparation course":"preparation",
"math score":"m_score",
"reading score": "r_score",
"writing score": "w_score"}, inplace = True)

# feature engineering on the data to visualize and solve the dataset more accurately
df['total_score'] = (df['m_score'] + df['r_score'] + df['w_score'])/3
df.head()

### Verifichiamo la presenza di righe nulle:

In [ ]:
df.isnull().sum()

### Osserviamo la correlazione delle features (numeriche):

In [ ]:
# start data exploration
df.corr()

### Osserviamo delle statistiche riassuntive sulle features numeriche:

In [ ]:
df.describe()

### Visualizziamo il numero di studenti per ogni gruppo:

In [ ]:
df.groupby(by="ethnicity").size().plot.barh()
plt.show()

Notiamo che la distribuzione dei gruppi è abbastanza eterogenea, e possiamo individuare due **minoranze**, gruppo A e gruppo E, che sono rappresentate per un numero di elementi minore della metà del gruppo predominante

### Visualizziamo il numero di studenti per ogni genere

In [ ]:
df.groupby(by="gender").size().plot.bar()
plt.show()

In [ ]:
df.groupby("ethnicity").gender.value_counts().plot.bar()
# non mi piace, vorrei farlo come quello sotto ma mi sto confondendo!
plt.show()

### Visualizziamo il numero di studenti per ogni genere all'interno di ogni gruppo

In [ ]:
f_filter = df["gender"] == "female"
females = df[f_filter].groupby(["ethnicity"]).size()
females = females.reset_index(name="female")

m_filter = df["gender"] == "male"
males = df[m_filter].groupby(["ethnicity"]).size()
males = males.reset_index(name="male")

genders_df = males.join(females,rsuffix='_drop').drop(columns="ethnicity_drop").set_index("ethnicity")
# grafico che voglio ma fa schifo ottenuto così
genders_df.plot.bar()
plt.xticks(rotation=0)
plt.show()

### Il gruppo C è quello più numeroso, ma anche quello con il rapporto più grande di presenza femminile

### Compariamo il numero di studenti che hanno una certa *Parental Education*:

In [ ]:
# visualizing the differnt parental education levels

df['parent_education'].value_counts()
df['parent_education'].value_counts().plot.bar()
plt.title('Comparison of Parental Education')
plt.xlabel('Titolo di studio')
plt.ylabel('Numero')
plt.xticks(rotation=45)

plt.show()

### Come varia la distribuzione dei voti al variare del livello di educazione dei genitori?

In [ ]:
# Come varia la distribuzione dei voti al variare del livello di educazione dei genitori?"
sns.set(rc={'figure.figsize':(20,7)})
fig, axs = plt.subplots(ncols=3)

sns.barplot(x = "parent_education", y = "w_score",  data = df, ax=axs[0])
sns.barplot(x = "parent_education", y = "r_score",  data = df, ax=axs[1])
sns.barplot(x = "parent_education", y = "m_score",  data = df, ax=axs[2])
for ax in axs:
    ax.tick_params(labelrotation=45)
    ax.tick_params(labelsize=12)
plt.show()


### Per i figli di laureati c'è una sostanziale predominanza, che si attenua leggermente nei voti di matematica

### Come varia tra i gruppi la distribuzione dei voti?

In [ ]:
# Come varia tra i gruppi la distribuzione dei voti?
sns.set(rc={'figure.figsize':(18,6)})
fig, axs = plt.subplots(ncols=3)

sns.barplot(x = "ethnicity", y = "w_score",  data = df, ax=axs[0])
sns.barplot(x = "ethnicity", y = "r_score",  data = df, ax=axs[1])
sns.barplot(x = "ethnicity", y = "m_score",  data = df, ax=axs[2])
for ax in axs:
    ax.tick_params(labelrotation=45,labelsize=12)
plt.show()

### Il gruppo E sembra comportarsi meglio degli altri in tutte le materie

### Come varia tra i generi la distribuzione dei voti?

In [ ]:
# Come varia tra i generi la distribuzione dei voti?
sns.set(rc={'figure.figsize':(18,6)})
fig, axs = plt.subplots(ncols=3)

sns.barplot(x = "gender", y = "w_score",  data = df, ax=axs[0])
sns.barplot(x = "gender", y = "r_score",  data = df, ax=axs[1])
sns.barplot(x = "gender", y = "m_score",  data = df, ax=axs[2])

plt.show()

### Le donne vengono superate dagli uomini solo in matematica!


### Come influisce il tipo di pasto sulla distribuzione dei voti?

In [ ]:
# Come influisce il tipo di pasto sulla distribuzione dei voti?
sns.set(rc={'figure.figsize':(18,6)})
fig, axs = plt.subplots(ncols=3)

sns.violinplot(x = "lunch", y = "w_score",  data = df, ax=axs[0])
sns.violinplot(x = "lunch", y = "r_score",  data = df, ax=axs[1])
sns.violinplot(x = "lunch", y = "m_score",  data = df, ax=axs[2])

plt.show()

### Gli studenti che assumono un pasto completo (supponiamo dunque i più ricchi) hanno una media dei voti sostanzialmente più alta in tutte le materie

### Come influisce il tipo di pasto sulla distribuzione dei voti?

In [ ]:
# Come influisce il tipo di pasto sulla distribuzione dei voti?
sns.boxplot(x = "ethnicity", y = "total_score",  data = df, hue="lunch")

plt.show()

### Il gruppo di minoranza A sembra essere quello che soffre di più l'influenza negativa del "free/reduced" lunch (la distanza dal centro del box del suo valore medio rispetto al suo margine superiore è più grande di tutte le altre)

### Osserviamo la ripartizione di studenti con pasto intero o ridotto all'interno dei vari gruppi

In [ ]:

df.groupby("lunch")["ethnicity"].value_counts()
# serve calcolare i rapporti tra standard e free per ogni gruppo, per vedere se i pasti gratis prevalgono in percentuale in qualche gruppo

### Osserviamo come sono ripartiti i diversi titoli di studio dei genitori all'interno dei vari gruppi

In [ ]:
df.groupby("lunch")["parent_education"].value_counts()

In [ ]:
# Data to plot
labels = 'group A', 'group B', 'group C', 'group D','group E'
sizes = df.groupby('ethnicity')['r_score'].mean().values
explode = (0.1, 0, 0, 0,0)  # explode 1st slice
 
# Plot
#plt.pie(sizes, explode=explode, labels=labels, colors=sns.color_palette("Set3"),
#autopct='%1.1f%%', shadow=True, startangle=140)
#plt.title('Reading Score for Every Ethnicity Mean')
#plt.axis('equal')
#plt.show()

# fare per le altre due materie

### Integriamo nel dataset di partenza alcune informazioni che ci serviranno per ulteriori statistiche:
* **status**: indica se lo studente ha superato tutte le materie (superando una certa soglia di voto scelta arbitrariamente)
* **grades**: una valutazione complessiva ricavata dalla media dei voti e basata sul sistema americano.

Gli studenti che non hanno superato tutte le materie ottengono una F come valutazione.

In [ ]:
# setting a passing mark for the students to pass on the three subjects individually
passmarks = 40

# creating a new column pass_math, this column will tell us whether the students are pass or fail
df['pass_math'] = np.where(df['m_score']< passmarks, 'Fail', 'Pass')
df['pass_reading'] = np.where(df['r_score']< passmarks, 'Fail', 'Pass')
df['pass_writing'] = np.where(df['w_score']< passmarks, 'Fail', 'Pass')

# checking which student is fail overall

df['status'] = df.apply(lambda x : 'Fail' if x['pass_math'] == 'Fail' or 
                           x['pass_reading'] == 'Fail' or x['pass_writing'] == 'Fail'
                           else 'pass', axis = 1)


### Mostriamo la distribuzione dei voti assegnati:

In [ ]:
# Assigning grades to the grades according to the following criteria :
# A: 90-100
# B: 80-89
# C: 70-79
# D: 60-69
# F: 0-59

def getgrade(total_score, status):
  if status == 'Fail':
    return 'F'
  if(total_score >= 90):
    return 'A'
  if(total_score >= 80):
    return 'B'
  if(total_score >= 70):
    return 'C'
  if(total_score >= 60):
    return 'D'
  else :
    return 'E'

df['grades'] = df.apply(lambda x: getgrade(x['total_score'], x['status']), axis = 1 )

axs = df['grades'].value_counts().plot.bar()
axs.tick_params(rotation = 0)
axs.set_xlabel('Voti',size=16)
plt.show()

### Verifichiamo adesso quanti ragazzi e ragazze decidono di completare il test di preparazione:

In [ ]:
plt.figure(figsize=(9,6))
sns.countplot(x = "preparation", hue="gender", data = df)

plt.show()

La distribuzione tra maschi e femmine rispecchia quella del dataset, tuttavia si evince che soltanto poco piu' della meta' degli studenti, indipendentemente dal genere, completa la preparazione al test

### Verifichiamo l'impatto della preparazione ai test sugli score

In [ ]:
fig, axs = plt.subplots(ncols=3)
colors = [ "amber", "windows blue","dusty purple","faded green", ]
color = sns.xkcd_palette(colors)

sns.barplot(x = "preparation", y = "w_score", hue="gender", data = df, ax=axs[0],palette=color)
sns.barplot(x = "preparation", y = "r_score",  hue="gender", data = df, ax=axs[1],palette=color[2:])
sns.barplot(x = "preparation", y = "m_score",  hue="gender", data = df, ax=axs[2])
plt.show()

### Ragazzi e ragazze, piu' o meno con la stessa influenza, risentono della mancanza di preparazione al test, che li porta ad ottenere punteggi inferiori di circa 5-10 punti rispetto a chi li completa

In [ ]:
sns.catplot(y="parent_education", hue="ethnicity", col="preparation", data=df, kind="count")
plt.show()

In questa analisi definiamo il concetto di **controtendenza** in questo modo:
>il numero di studenti che scelgono di preparsi al test e' uguale a quello di studenti che non si prepara al test (in controtendenza dunque al rapporto generale).

Nel gruppo B
* Sebbene tra i figli di laureati triennali, che per la maggior parte preferiscono non prepararsi, abbiano all'incirca gli stessi risultati (poco inferiori) dei **figli di laureati magistrali**, questi ultimi sono in **controtendenza**
* Nei restanti livelli di istruzione, la situazione rispecchia quella generale, ma c'e' un'ulteriore anomalia tra **i figli di persone che non hanno completato gli studi**, infatti anche questi sono in **controtendenza**

Nel gruppo C
* **Controtendenza** dei ragazzi figli di **laureati triennali**.
* Controtendenza meno decisa dei ragazzi figli di laureati magistrali, e di genitori che non hanno completato gli studi. 

Nel gruppo A
* **Tutti i ragazzi** figli di laureati magistrali **decidono di non prepararsi al test**.

Nel gruppo D
* Rispecchia l'andamento generale senza particolari anomalie.

Nel gruppo E
* Marcata controtendenza dei ragazzi figli di **laureati triennali**, i quali **per la maggior parte decidono di prepararsi al test**.
* **Controtendenza** dei ragazzi figli di genitori che **non hanno completato gli studi**.
* Controtendenza meno decisa dei ragazzi figli di genitori con diploma associato.  